In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

C:\Users\ranje\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 00:23:55,021 INFO: Initializing external client
2025-03-05 00:23:55,022 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:23:55,906 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214704
Fetching data from 2025-02-04 05:23:55.020062+00:00 to 2025-03-05 04:23:55.020062+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.01s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 06:00:00+00:00,2,0
1,2025-02-04 07:00:00+00:00,2,0
2,2025-02-04 08:00:00+00:00,2,0
3,2025-02-04 09:00:00+00:00,2,0
4,2025-02-04 10:00:00+00:00,2,0
...,...,...,...
172934,2025-03-04 18:00:00+00:00,263,132
172935,2025-03-04 19:00:00+00:00,263,99
172936,2025-03-04 20:00:00+00:00,263,86
172937,2025-03-04 21:00:00+00:00,263,76


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172939 entries, 0 to 175197
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         172939 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  172939 non-null  int32                  
 2   rides               172939 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172939 entries, 0 to 175197
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         172939 non-null  datetime64[us]
 1   pickup_location_id  172939 non-null  int32         
 2   rides               172939 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,99,65,6,168,44,30,30,24,15,3,...,54,161,203,1,34,9,5,36,114,2025-02-05 17:00:00
1,0,2,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,165,2025-03-04 12:00:00
2,1,0,3,1,4,0,1,0,2,0,...,0,0,2,0,1,0,0,0,93,2025-02-14 00:00:00
3,112,11,127,41,81,35,48,104,95,88,...,42,14,14,61,39,141,5,123,263,2025-02-21 23:00:00
4,0,0,0,0,0,0,2,0,0,0,...,2,0,0,1,0,0,0,0,241,2025-02-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,7,8,0,0,2,1,1,3,1,1,...,1,1,3,62,9,2,7,4,4,2025-02-12 17:00:00
247,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,21,2025-02-12 01:00:00
248,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,94,2025-02-04 19:00:00
249,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,185,2025-02-14 00:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 00:24:02,433 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 00:24:02,437 INFO: Initializing external client
2025-03-05 00:24:02,437 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:24:03,164 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214704
Fetching data from 2025-02-04 05:24:02.433585+00:00 to 2025-03-05 04:24:02.433585+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.43s) 


In [12]:
current_date

Timestamp('2025-03-05 05:24:02.433585+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 06:00:00
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 06:00:00
2,2,1,11,7,3,2,3,2,1,7,...,1,1,4,4,3,0,0,1,4,2025-03-04 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 06:00:00
4,2,2,1,2,3,2,3,1,2,4,...,1,0,0,1,0,0,0,0,7,2025-03-04 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,259,2025-03-04 06:00:00
247,1,4,2,0,1,1,1,0,1,1,...,0,1,0,1,0,0,0,1,260,2025-03-04 06:00:00
248,0,4,5,16,21,34,25,31,37,28,...,8,10,1,0,0,1,1,1,261,2025-03-04 06:00:00
249,63,126,177,156,102,81,88,81,79,97,...,14,9,2,2,1,4,4,16,262,2025-03-04 06:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 00:24:10,234 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 00:24:10,238 INFO: Initializing external client
2025-03-05 00:24:10,239 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:24:10,986 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214704


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,-0.0
1,3,0.0
2,4,2.0
3,6,0.0
4,7,-1.0
...,...,...
246,259,0.0
247,260,-0.0
248,261,30.0
249,262,82.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 186, 236, 162, 237, 230, 100, 161, 239, 234])